## Basic DNN model to for testing #no prepocessing done except normalisation

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from datetime import datetime, date
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
import matplotlib.pyplot as plt

In [27]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/data.csv")
df

,your_type_of_document,your_nationality,your_gender,your_birthdate,your_foreigner,period,your_consecutive,your_marital_status,your_student,your_country_resides,your_have_ethnicity,your_dept_resides,your_dept_code_resides,your_municipality_resides,your_municipality_code_resides,your_high_school_degree,your_tuition_cost_university,your_tuition_pymt_scholarship,your_tuition_pymt_credit,your_tuition_pymt_parents,your_tuition_pymt_yourself,fami_current_household_type,fami_is_the_head_family,fami_econ_dependents,fami_father_education,fami_mother_education,fami_father_occupation,fami_mother_occupation,fami_housing_stratum,fami_internet,fami_computer,fami_automobile,fami_num_books,inst_institution_code,inst_institution_name,your_prgm_academic,group_reference,your_prgm_municipality_code,your_prgm_municipality,your_prgm_department,...,fami_social_stratum_sisben,fami_house_members,fami_house_rooms,fami_house_floor,fami_washer,fami_microwave,fami_oven,fami_dvd,fami_monthly_family_income,your_currently_working,your_background,cole_saber_11_icfes_code,cole_saber_11_icfes_dane,cole_saber_11_icfes_name,cole_type,cole_nature,cole_calendar,cole_bilingual,cole_character,cole_dane_code_campus,cole_name_campus,cole_main_campus,cole_area_location,cole_journey,cole_municipality_code_location,cole_municipality_location,cole_dept_code_location,cole_dept_location,score_language_saber_11,score_mathematics_saber_11,score_biology_saber_11,score_chemistry_saber_11,score_physics_saber_11,score_social_science_saber_11,score_philosophy_saber_11,score_english_saber_11,score_english_saber_11_category,optative_field_saber_11,score_optative_saber_11,optative_category_saber_11
0,TI,COLOMBIA,M,30/07/1996,NO,20134,EK201340233804,Single,STUDENT,COLOMBIA,No,BOYACA,15.0,GUATEQUE,15322.0,Technical Bachelor,Did not pay tuition,No,No,Si,No,It is habitual or permanent,No,None,Undergraduate Degree complete,Middle School complete,Profesional independiente,Profesional independiente,Stratum 3,Si,No,No,Unknown,9110,SERVICIO NACIONAL DE APRENDIZAJE-SENA,TECNOLOGÍA EN CONTABILIDAD Y FINANZAS,TECNOLOGICO EN ADMINISTRACIÓN Y TURISMO,11001,BOGOTÁ D.C.,BOGOTA,...,Not classified by SISBEN,Four,Four,"Polished wood, tile, tablet, marble, carpet",Si,Si,Si,Si,Unknown,No,No,5207.0,1.153220e+11,I.E. VALLE DE TENZA,MIXED,OFFICIAL,A,N,TECHNICAL/ACADEMIC,1.153220e+11,I.E. VALLE DE TENZA - SEDE PRINCIPAL,YES,URBAN,MORNING,15322.0,GUATEQUE,15.0,BOYACA,32.0,43.0,39.0,42.0,33.0,33.0,29.0,38.0,A-,SCORE_INTERDISC_ENVIRONMENT,49.0,UNK
1,CC,COLOMBIA,M,13/04/1994,NO,20133,EK201330220754,Single,STUDENT,COLOMBIA,No,AMAZONAS,91.0,LETICIA,91001.0,Academic Bachelor,Did not pay tuition,No,No,Si,No,It is habitual or permanent,No,None,Middle School complete,Technical School complete,Empleado de nivel auxiliar o administrativo,Self-employed,Stratum 2,No,Si,Si,Unknown,2106,DIRECCION NACIONAL DE ESCUELAS-BOGOTÁ D.C.,TECNICO PROFESIONAL EN SERVICIO DE POLICIA,TECNOLOGICO EN MILITAR Y POLICIAL,11001,BOGOTÁ D.C.,BOGOTA,...,Not classified by SISBEN,Five,Three,"Cement, gravel, brick",Si,No,No,Si,Between 2 and less than 3 SMLV,"Yes, 20 hours or more per week",No,38919.0,1.910010e+11,I.E. INEM JOSE EUSTASIO RIVERA,MIXED,OFFICIAL,A,N,TECHNICAL/ACADEMIC,1.910010e+11,I.E. INEM JOSE EUSTASIO RIVERA - SEDE PRINCIPAL,YES,URBAN,MORNING,91001.0,LETICIA,91.0,AMAZONAS,48.0,64.0,52.0,52.0,54.0,44.0,47.0,51.0,A1,SCORE_DEEPEN_LANGUAGE,6.0,II
2,CC,COLOMBIA,F,08/12/1991,NO,20134,EK201340246502,Single,STUDENT,COLOMBIA,No,RISARALDA,66.0,DOSQUEBRADAS,66170.0,Technical Bachelor,Did not pay tuition,No,No,Si,No,It is habitual or permanent,No,None,Elementary school incomplete,Elementary school incomplete,Other activity or occupation,Home,Stratum 2,No,No,No,Unknown,9110,SERVICIO NACIONAL DE APRENDIZAJE-SENA,TECNOLOGÍA EN CONTABILIDAD Y FINANZAS,TECNOLOGICO EN ADMINISTRACIÓN Y TURISMO,66001,PEREIRA,RISARALDA,...,Level 2,Five,Three,"Cement, gravel, brick",Si,Si,Si,Si,Less than 1 SMLV,No,Si,61424.0,1.660010e+11,IE AQUILINO BEDOYA,MIXED,OFFICIAL,

In [26]:
df['optative_field_saber_11'].value_counts()

SCORE_INTERDISC_VIOLENCE_AND_SOC    57168
SCORE_INTERDISC_ENVIRONMENT         56887
SCORE_DEEPEN_LANGUAGE               30124
SCORE_DEEPEN_MATHEMATICS            28987
SCORE_DEEPEN_BIOLOGY                22555
SCORE_DEEPEN_SOCIAL_SCIENCE         16289
Name: optative_field_saber_11, dtype: int64

In [3]:
Y_encode, Y_name = pd.factorize(df['optative_field_saber_11'])

In [4]:
X = df.drop(['your_student', 'optative_field_saber_11'], axis=1)
#your_student has only single categorical value

In [5]:
list_of_columns = X.columns

In [6]:
#distribution normalisation using mean adn standard deviation
def normalise_encode(en):
    return ( en - en.mean() ) / en.std()

#normalising data between [0 - 1]
def normalise_binary(en):
    up = en - en.min()
    down = en.max() - en.min()
    return (up/down) + 0.0001 

In [7]:
# converting categorical data into embeddings ( numerical values ) followed by normalising data
encode = {}
name = {}
X_train = pd.DataFrame()
for col in list_of_columns:
    if X[col].dtype == 'float64' or X[col].dtype == 'float32' or X[col].dtype == 'int':
        X_train[col] = normalise_binary(X[col])
    else:
        encode[col], name[col] = pd.factorize(X[col])
        X_train[col] = normalise_binary(encode[col])
    

In [8]:
#one hot encode class labels
Y_train = tf.one_hot(Y_encode,6)

In [21]:
model = None
model = Sequential()
model.add(Dense(100, input_dim=96, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(6, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 100)               9700      
_________________________________________________________________
batch_normalization_9 (Batch (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 100)               10100     
_________________________________________________________________
batch_normalization_10 (Batc (None, 100)               400       
_________________________________________________________________
dense_12 (Dense)             (None, 100)               10100     
_________________________________________________________________
batch_normalization_11 (Batc (None, 100)              

In [22]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss = tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [19]:
#this performance reached after 186 epochs (trained twice)
model.fit(X_train, Y_train, epochs=100, batch_size=128, validation_split=0.5)

Epoch 1/100
829/829 [==============================] - 12s 14ms/step - loss: 0.5434 - accuracy: 0.7695 - val_loss: 0.8625 - val_accuracy: 0.6477
Epoch 2/100
829/829 [==============================] - 12s 14ms/step - loss: 0.5358 - accuracy: 0.7718 - val_loss: 1.0397 - val_accuracy: 0.5990
Epoch 3/100
829/829 [==============================] - 12s 14ms/step - loss: 0.5343 - accuracy: 0.7742 - val_loss: 0.7976 - val_accuracy: 0.6742
Epoch 4/100
829/829 [==============================] - 12s 14ms/step - loss: 0.5370 - accuracy: 0.7723 - val_loss: 0.8602 - val_accuracy: 0.6528
Epoch 5/100
829/829 [==============================] - 12s 14ms/step - loss: 0.5317 - accuracy: 0.7748 - val_loss: 0.7822 - val_accuracy: 0.6782
Epoch 6/100
829/829 [==============================] - 12s 14ms/step - loss: 0.5298 - accuracy: 0.7752 - val_loss: 0.7983 - val_accuracy: 0.6762
Epoch 7/100
829/829 [==============================] - 12s 14ms/step - loss: 0.5292 - accuracy: 0.7772 - val_loss: 0.8460 - val_ac

KeyboardInterrupt: ignored

In [23]:
#testing out validation split after reinitiating model to avoid retraining

model.fit(X_train, Y_train, epochs=100, batch_size=128, validation_split=0.2)

Epoch 1/100
1326/1326 [==============================] - 19s 13ms/step - loss: 1.0325 - accuracy: 0.4891 - val_loss: 0.8878 - val_accuracy: 0.5738
Epoch 2/100
1326/1326 [==============================] - 16s 12ms/step - loss: 0.8918 - accuracy: 0.5793 - val_loss: 0.8144 - val_accuracy: 0.6326
Epoch 3/100
1326/1326 [==============================] - 16s 12ms/step - loss: 0.8334 - accuracy: 0.6202 - val_loss: 0.7887 - val_accuracy: 0.6511
Epoch 4/100
1326/1326 [==============================] - 16s 12ms/step - loss: 0.8117 - accuracy: 0.6338 - val_loss: 0.7754 - val_accuracy: 0.6536
Epoch 5/100
1326/1326 [==============================] - 16s 12ms/step - loss: 0.8027 - accuracy: 0.6395 - val_loss: 0.7614 - val_accuracy: 0.6608
Epoch 6/100
1326/1326 [==============================] - 16s 12ms/step - loss: 0.7928 - accuracy: 0.6455 - val_loss: 0.7660 - val_accuracy: 0.6594
Epoch 7/100
1326/1326 [==============================] - 16s 12ms/step - loss: 0.7863 - accuracy: 0.6480 - val_loss: 0